In [1]:
import pandas as pd
import numpy as np
from numpy import linalg
from sklearn.model_selection import StratifiedKFold
import nltk
import pickle
from nltk import pos_tag
from scipy.sparse import hstack, vstack
from scipy import spatial
from sklearn.datasets import dump_svmlight_file,load_svmlight_file
from sklearn.decomposition import TruncatedSVD
import xgboost as xgb
import gensim
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import multiprocessing
import gc
import logging
# 显示cell运行时长
%load_ext klab-autotime

In [2]:
colnames = ["query_id","query","query_title_id","title","label"]

# load dict
save_path = "/home/kesci/work/word2vec/wordvectors.kv"
w2v_dict = KeyedVectors.load(save_path, mmap='r')
len(w2v_dict["1427"])
################ avg word2vec 表征 sentence vector ##############
################ avg word2vec 表征 sentence vector ##############
################ avg word2vec 表征 sentence vector ##############

128

time: 3.7 s


In [3]:
# load idf——weight
q_idf_weight = pickle.load(open("/home/kesci/work/idf/idf_weight_query.pkl","rb"))
t_idf_weight = pickle.load(open("/home/kesci/work/idf/idf_weight_title.pkl","rb"))
################ tf-idf加权 word2vec 表征 sentence vector ##############
################ tf-idf加权 word2vec 表征 sentence vector ##############
################ tf-idf加权 word2vec 表征 sentence vector ##############

time: 1.82 s


In [4]:
def cosine_sim(query, title):
    l2_query = linalg.norm(query,ord=2,axis=1)
    l2_title = linalg.norm(title,ord=2,axis=1)
    eudistance = linalg.norm(query-title,ord=2,axis=1) # 欧式距离 L2
    manhattan = linalg.norm(query-title,ord=1,axis=1) # 曼哈顿距离 L1
    chebyshev = linalg.norm(query-title,ord=np.inf,axis=1) # 切比雪夫距离 Max|x1-x2|
    chebyshev_ = linalg.norm(query-title,ord=-np.inf,axis=1) # Min |x1-x2|
    my_sqrt_distance = np.sum(np.sqrt(np.absolute(query-title)),axis=1) # sqrt|x1-x2|
    sim = np.sum(query * title,axis=1)/(l2_query * l2_title)
    
    combine_sim = np.hstack((sim.reshape(-1,1),
                             eudistance.reshape(-1,1),
                             manhattan.reshape(-1,1),
                             chebyshev.reshape(-1,1),chebyshev_.reshape(-1,1),
                             my_sqrt_distance.reshape(-1,1)
                            ))
    return combine_sim

#### 1w大小的数据  计算一次
def concat_sentence(origin_data):
    q_vstack = ""
    t_vstack = ""
    for row in origin_data.values:
        # 1.query
        words = row[1].split(" ")
        M = []
        for w in words:
            try:
                M.append(w2v_dict[w] * q_idf_weight[w])
            except:
                continue
        M = np.array(M)
        if M.shape[0] != 0:
            q_sentence_vec = M.sum(axis=0) / M.shape[0]
        else:
            q_sentence_vec = np.full(word_dim,np.finfo(np.float32).eps)
        if q_vstack == "":
            q_vstack = q_sentence_vec
        else:
             q_vstack = np.vstack((q_vstack,q_sentence_vec))
        
        # 2.title
        words = row[3].split(" ")
        M = []
        for w in words:
            try:
                M.append(w2v_dict[w] * t_idf_weight[w])
            except:
                continue
        M = np.array(M)
        if M.shape[0] != 0:
            t_sentence_vec = M.sum(axis=0) / M.shape[0]
        else:
            t_sentence_vec = np.full(word_dim,np.finfo(np.float32).eps)
        if t_vstack == "":
            t_vstack = t_sentence_vec
        else:
             t_vstack = np.vstack((t_vstack,t_sentence_vec))
    return q_vstack,t_vstack

time: 4.92 ms


In [6]:
# ############## 后500w 训练集数据 ############
# feature_path = "/home/kesci/work/similarity_feature/train_idf/"
# train_data = "/home/kesci/input/bytedance/first-round/train.csv"
# # sentece vector of (query-title)
# sentence_vector_path = feature_path+"sentence_vector.csv"
# sentence_vec_fout = open(sentence_vector_path,"w")
# # 6dim similarity feature
# consine_feature_path = feature_path+"consine_sentence_sim.csv"
# sim_qt_fout = open(consine_feature_path,"w")

# samples = 100000000
# chunksize = 10000
# skip_num = int(samples/chunksize) - 500
# print(skip_num)

# for i in range(int(500)):
#     df = pd.read_csv(train_data, names=colnames, header=None,skiprows=chunksize*(skip_num+i),nrows=chunksize,lineterminator="\n")
#     # 1.get sentece_vector & consine sim
#     q_vstack,t_vstack = concat_sentence(df)
#     combine_sim = cosine_sim(q_vstack,t_vstack)
#     # 2.save
#     np.savetxt(sentence_vec_fout,q_vstack - t_vstack,delimiter=",",fmt="%.5f")
#     np.savetxt(sim_qt_fout,combine_sim,delimiter=",",fmt="%.5f")

#     # 3.print log
#     if (i+1) % 100 == 0:
#         print((q_vstack - t_vstack).shape,combine_sim.shape)
#     # # 4.gc
#     # del df,q_vstack,t_vstack
#     # gc.collect()

# sentence_vec_fout.flush()
# sim_qt_fout.flush()


9500


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:37: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:55: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


(10000, 128)
(10000, 128)
(10000, 128)
(10000, 128)
(10000, 128)
time: 4h 52min 5s


In [7]:
!cat /home/kesci/work/similarity_feature/train_idf/consine_sentence_sim.csv | wc -l
!cat /home/kesci/work/similarity_feature/train_idf/sentence_vector.csv | wc -l
!cat /home/kesci/work/similarity_feature/train_idf/consine_sentence_sim.csv | head -n 1
!cat /home/kesci/work/similarity_feature/train_idf/sentence_vector.csv | head -n 1

79910
80000
time: 1.23 s


In [ ]:
########################  验证集 validation #######################
########################  验证集 validation #######################
########################  验证集 validation #######################

In [9]:
############## 1000w-500w 验证集数据 ############
word_dim = 128
feature_path = "/home/kesci/work/similarity_feature/valid_idf/"
train_data = "/home/kesci/input/bytedance/first-round/train.csv"
# sentece vector of (query-title)
sentence_vector_path = feature_path+"sentence_vector.csv"
sentence_vec_fout = open(sentence_vector_path,"w")
# 6dim similarity feature
consine_feature_path = feature_path+"consine_sentence_sim.csv"
sim_qt_fout = open(consine_feature_path,"w")

samples = 100000000
chunksize = 10000
skip_num = int(samples/chunksize) - 1000
print(skip_num)

for i in range(int(500)):
    df = pd.read_csv(train_data, names=colnames, header=None,skiprows=chunksize*(skip_num+i),nrows=chunksize,lineterminator="\n")
    # 1.get sentece_vector & consine sim
    q_vstack,t_vstack = concat_sentence(df)
    combine_sim = cosine_sim(q_vstack,t_vstack)
    # 2.save
    np.savetxt(sentence_vec_fout,q_vstack - t_vstack,delimiter=",",fmt="%.5f")
    np.savetxt(sim_qt_fout,combine_sim,delimiter=",",fmt="%.5f")

    # 3.print log
    if (i+1) % 100 == 0:
        print((q_vstack - t_vstack).shape,combine_sim.shape)
    # # 4.gc
    # del df,q_vstack,t_vstack
    # gc.collect()

sentence_vec_fout.flush()
sim_qt_fout.flush()


9000


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:37: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:55: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


(10000, 128) (10000, 6)
(10000, 128) (10000, 6)
(10000, 128) (10000, 6)
(10000, 128) (10000, 6)
(10000, 128) (10000, 6)
time: 5h 4min 34s


In [ ]:
!cat /home/kesci/work/similarity_feature/valid_idf/consine_sentence_sim.csv | wc -l
!cat /home/kesci/work/similarity_feature/valid_idf/sentence_vector.csv | wc -l
!cat /home/kesci/work/similarity_feature/valid_idf/consine_sentence_sim.csv | head -n 1
!cat /home/kesci/work/similarity_feature/valid_idf/sentence_vector.csv | head -n 1

In [ ]:
######################  测试集 Test ##########################
######################  测试集 Test ##########################
######################  测试集 Test #########################

In [ ]:
# ############## 测试集数据 ############
# feature_path = "/home/kesci/work/similarity_feature/test/"
# test_data = "/home/kesci/input/bytedance/first-round/test.csv"
# # sentece vector of (query-title)
# sentence_vector_path = feature_path+"sentence_vector.csv"
# sentence_vec_fout = open(sentence_vector_path,"w")
# # 6dim similarity feature
# consine_feature_path = feature_path+"consine_sentence_sim.csv"
# sim_qt_fout = open(consine_feature_path,"w")

# chunksize = 10000

# for i in range(int(500)):
#     df = pd.read_csv(test_data, names=colnames, header=None,skiprows=chunksize*i,nrows=chunksize,lineterminator="\n")
#     # 1.get sentece_vector & consine sim
#     q_vstack,t_vstack = concat_sentence(df)
#     combine_sim = cosine_sim(q_vstack,t_vstack)
#     # 2.save
#     np.savetxt(sentence_vec_fout,q_vstack - t_vstack,delimiter=",",fmt="%.5f")
#     np.savetxt(sim_qt_fout,combine_sim,delimiter=",",fmt="%.5f")

#     # 3.print log
#     if (i+1) % 100 == 0:
#         print((q_vstack - t_vstack).shape,combine_sim.shape)
#     # # 4.gc
#     # del df,q_vstack,t_vstack
#     # gc.collect()

# sentence_vec_fout.flush()
# sim_qt_fout.flush()


In [8]:
########################  query_word & sentence similarity 的计算 ###################
########################          min,max,avg,median,std          ###################
########################  query_word & sentence similarity 的计算 ###################
########################          min,max,avg,median,std          ###################
########################  query_word & sentence similarity 的计算 ###################
########################          min,max,avg,median,std          ###################
########################  query_word & sentence similarity 的计算 ###################
########################          min,max,avg,median,std          ###################

time: 596 µs


In [9]:
##################################################
##########计算word 和title_sentence sim
##################################################
word_dim = 128
from scipy import spatial
def cosine_sim(A, B):
    return 1 - spatial.distance.cosine(A, B)

def word_sentence_sim(origin_data):
    sim_vstack = ""
    for row in origin_data.values:        
        # 1. title sentence vector
        words = nltk.SpaceTokenizer().tokenize(row[3])
        t_sentence_vec = np.full(word_dim,np.finfo(np.float32).eps)
        M = []
        for w in words:
            try:
                M.append(w2v_dict[w])
            except:
                continue
        M = np.array(M)
        if M.shape[0] != 0:
            t_sentence_vec = M.sum(axis=0) / M.shape[0]
        
        # 2.get max,min,avg,median,std
        words = nltk.SpaceTokenizer().tokenize(row[1])
        M = []
        for w in words:
            try:
                A = w2v_dict[w]
                M.append(cosine_sim(A,t_sentence_vec))
            except:
                continue
        M = np.array(M)
        if M.shape[0] == 0:
            M = np.zeros(word_dim)
        sim_hstack = np.hstack(( 
                            np.min(M),np.max(M),
                            np.average(M),np.median(M),np.std(M)
                               ))         
        if sim_vstack == "":
            sim_vstack = sim_hstack.reshape(1,-1)
        else:
            sim_vstack = np.vstack((sim_vstack,sim_hstack.reshape(1,-1)))
    return sim_vstack

time: 5.44 ms


In [ ]:
######################### 训练集数据 ########################
######################### 训练集数据 ########################
######################### 训练集数据 ########################

In [10]:
############## 后500w 训练集数据 ############
feature_path = "/home/kesci/work/similarity_feature/train/"
train_data = "/home/kesci/input/bytedance/first-round/train.csv"
# # 5dim word_similarity feature
consine_feature_path = feature_path+"consine_word_sim.csv"
sim_qt_fout = open(consine_feature_path,"w")

samples = 100000000
chunksize = 10000
skip_num = int(samples/chunksize) - 500
print(skip_num)

for i in range(int(500)):
    df = pd.read_csv(train_data, names=colnames, header=None,skiprows=chunksize*i,nrows=chunksize,lineterminator="\n")
    combine_sim = word_sentence_sim(df)
    np.savetxt(sim_qt_fout,combine_sim,delimiter=",",fmt="%.5f") 
    # print log
    if (i+1) % 100 == 0:
        print(combine_sim.shape)

sim_qt_fout.flush()


9500


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:41: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


(10000, 5)
(10000, 5)
(10000, 5)
(10000, 5)
(10000, 5)
time: 1h 7min 9s


In [ ]:
######################### 验证集数据 ########################
######################### 验证集数据 ########################
######################### 验证集数据 ########################

In [ ]:
# ############## 1000w-500w 验证集数据 ############
# feature_path = "/home/kesci/work/similarity_feature/valid/"
# train_data = "/home/kesci/input/bytedance/first-round/train.csv"
# # # 5dim word_similarity feature
# consine_feature_path = feature_path+"consine_sentence_sim.csv"
# sim_qt_fout = open(consine_feature_path,"w")

# samples = 100000000
# chunksize = 10000
# skip_num = int(samples/chunksize) - 1000
# print(skip_num)

# for i in range(int(500)):
#     df = pd.read_csv(train_data, names=colnames, header=None,skiprows=chunksize*i,nrows=chunksize,lineterminator="\n")
#     combine_sim = word_sentence_sim(df)
#     np.savetxt(sim_qt_fout,combine_sim,delimiter=",",fmt="%.5f") 
#     # print log
#     if (i+1) % 100 == 0:
#         print(combine_sim.shape)

# sim_qt_fout.flush()


In [ ]:
######################### 测试集数据 ########################
######################### 测试集数据 ########################
######################### 测试集数据 ########################

In [ ]:

# feature_path = "/home/kesci/work/similarity_feature/test/"
# test_data = "/home/kesci/input/bytedance/first-round/test.csv"
# # 5dim word_similarity feature
# consine_feature_path = feature_path+"consine_word_sim.csv"
# sim_qt_fout = open(consine_feature_path,"w")

# chunksize = 10000

# for i in range(int(500)):
#     df = pd.read_csv(test_data, names=colnames, header=None,skiprows=chunksize*i,nrows=chunksize,lineterminator="\n")
#     combine_sim = word_sentence_sim(df)
#     np.savetxt(sim_qt_fout,combine_sim,delimiter=",",fmt="%.5f") 
#     # print log
#     if (i+1) % 100 == 0:
#         print(combine_sim.shape)

# sim_qt_fout.flush()
